# Tensorflow学习

In [1]:
import tensorflow as tf

In [2]:
# 创建一张图,上下文环境
g=tf.Graph()

print(g)

with g.as_default():
    c=tf.constant(11.0)
    print(c.graph)

In [3]:
# 实现一个加法运算
a=tf.constant(5.0)
b=tf.constant(6.0)
sum1=tf.add(a,b)

In [5]:
tf.get_default_graph()

In [7]:
# 只能运行一个图结构
with tf.Session() as sess:
    print(sess.run(sum1))

11.0


In [7]:
a.graph

In [8]:
sum1.graph

说明这张图在同一块内存当中

In [11]:
# 可以在会话中指定一张图
with tf.Session(graph=g) as sess:
    print(sess.run(c))

11.0


sess.run()相当于启动某个图

In [14]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    print(sess.run(sum1))

11.0


In [18]:
# placeholder相当于一个占位符,feed_dict是一个字典
# plt=tf.placeholder(tf.float32,[2,3])
plt=tf.placeholder(tf.float32,[None,3])
print(plt)
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    print(sess.run(plt,feed_dict={plt:[[1,2,3],[4,5,6]]}))
    print('*'*20)
    print(a.shape)
    print('*'*20)
    print(plt.shape)
    print('*'*20)
    print(a.name)
    print('*'*20)
    print(a.op)

Tensor("Placeholder_2:0", shape=(?, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]
********************
()
********************
(?, 3)
********************
Const:0
********************
name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 5.0
    }
  }
}



In [19]:
a.shape

TensorShape([])

In [20]:
plt.shape

TensorShape([Dimension(None), Dimension(3)])

In [21]:
a.name

'Const:0'

In [22]:
a.op

<tf.Operation 'Const' type=Const>

0维：()  
1维：(5)  
2维：(5,6)  
3维：(2,3,4)

In [23]:
# 设置形状
plt=tf.placeholder(tf.float32,[None,2])
plt

<tf.Tensor 'Placeholder_3:0' shape=(?, 2) dtype=float32>

In [25]:
plt.set_shape([4,2])
plt

<tf.Tensor 'Placeholder_3:0' shape=(4, 2) dtype=float32>

In [ ]:
# 对于静态形状来说，一旦张量形状固定了，不能再次设置形状 不能跨纬度修改
# plt.set_shape([3,2])
# ValueError: Shapes (4, 2) and (3, 2) are not compatible

In [28]:
# 动态形状，改变时元素一定要匹配
plt_reshape=tf.reshape(plt,[2,4])
plt_reshape

<tf.Tensor 'Reshape_1:0' shape=(2, 4) dtype=float32>

张量操作，生成张量

In [8]:
# 开启交互式会话
tf.InteractiveSession()

In [33]:
zero=tf.zeros([3,4],tf.float32)
zero.eval()

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)

In [37]:
ones=tf.ones([3,4],tf.float32)
ones.eval()

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)

In [35]:
constant=tf.constant([3,4],tf.float32)
constant.eval()

array([3., 4.], dtype=float32)

In [4]:
test=tf.cast([[1,2,3],[4,4.5,6]],tf.int32)
test.eval()

array([[1, 2, 3],
       [4, 4, 6]])

In [6]:
# 列表的合并
a=[[7,8,5],[6,9,1]]
b=[[4,5,6],[1,2,3]]
c=tf.concat([a,b],axis=0)
c.eval()

array([[7, 8, 5],
       [6, 9, 1],
       [4, 5, 6],
       [1, 2, 3]])

# Tensorflow里的变量

In [15]:
a=tf.constant([1,2,3,4,5],name='a')
var=tf.Variable(tf.random_normal([2,3],mean=0.0,stddev=1.0),name='var')

In [16]:
print(a,var)

Tensor("a:0", shape=(5,), dtype=int32) Tensor("var/read:0", shape=(2, 3), dtype=float32)


In [11]:
# 必须做一部现实的初始化
init_op=tf.global_variables_initializer()

In [5]:
with tf.Session() as sess:
    # 必须运行初始化op
    sess.run(init_op)
    print(sess.run([a,var]))

[array([1, 2, 3, 4, 5]), array([[ 0.16678654,  1.8401545 , -1.8440102 ],
       [-0.10111339,  0.25699118,  0.44848678]], dtype=float32)]


### 总结  
1、变量能持久化保存，普通op不能  
2、当定义一个变量op时候必须在会话中初始化  
3、name参数：在tensorboard使用的时候显示名字，可以让相同op进行区分

# Tensorflow的可视化

In [14]:
with tf.Session() as sess:
    # 必须运行初始化op
    sess.run(init_op)
    # 把程序的图写入指定路径
    filewrite=tf.summary.FileWriter('Data/tem/',graph=sess.graph)
    print(sess.run([a,var]))

[array([1, 2, 3, 4, 5]), array([[-0.3551535 ,  0.4698463 , -2.7322927 ],
       [-1.0268147 , -1.3277575 , -0.52979785]], dtype=float32)]


启动服务命令  
tensorboard --logdir=C:\Users\17575\VSCode\机器学习\Data\tem  
注意：目录并不需要加引号, logs 中有多个event 时，会生成scalar 的对比图，但 graph 只会展示最新的结果